In [1]:
from IPython.core.display import display,HTML

display(HTML("<style>.container { width: 90% !important;}</style>"))

/var/folders/s2/bjtyvybn0kv3rwjkn5xht9_m0000gn/T/ipykernel_3226/2814205877.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display,HTML


In [2]:
import pandas as pd
import govt_func as gf
import datetime
from imp import reload

import numpy as np
from dateutil.relativedelta import *

# Import Data

In [3]:
nonfreq_uri = "data/non-frequent_gov_sub.xlsx"
df_nonfreq = pd.read_excel(nonfreq_uri)
profit_uri = "data/profit.xlsx"
df_profit = pd.read_excel(profit_uri)
df_nonfreq = df_nonfreq.drop("证券名称", axis=1)
df_profit = df_profit.drop("证券名称", axis=1)
df_nonfreq = df_nonfreq.rename(columns={"CITICS_INDUSTRY": "INDUSTRY"})
df_profit = df_profit.rename(columns={"CITICS_INDUSTRY": "INDUSTRY"})
df_nonfreq = df_nonfreq.dropna(axis=0, subset="INDUSTRY")
df_profit = df_profit.dropna(axis=0, subset="INDUSTRY")
df_nonfreq = df_nonfreq.drop(
    df_nonfreq.iloc[:, 2:].filter(like="Y", axis=1).columns, axis=1
)  # exclude annual statement
df_profit = df_profit.drop(
    df_profit.iloc[:, 2:].filter(like="Y", axis=1).columns, axis=1
)  # exclude annual statement
df_profit

,SECU_CODE,INDUSTRY,2011Q1,2011Q2,2011Q3,2012Q1,2012Q2,2012Q3,2013Q1,2013Q2,...,2019Q3,2020Q1,2020Q2,2020Q3,2021Q1,2021Q2,2021Q3,2022Q1,2022Q2,2022Q3
0,000001.SZ,银行,3.069099e+09,6.065846e+09,9.924443e+09,4.477351e+09,8.878649e+09,1.341211e+10,4.718000e+09,9.906000e+09,...,3.067600e+10,1.095800e+10,1.758700e+10,2.882600e+10,1.287400e+10,2.234100e+10,3.678700e+10,1.618400e+10,2.778300e+10,4.611200e+10
1,000002.SZ,房地产,1.674455e+09,4.451809e+09,5.650196e+09,2.058143e+09,6.048672e+09,8.165983e+09,2.394886e+09,7.133412e+09,...,4.182479e+10,4.795679e+09,2.645044e+10,4.183800e+10,4.108071e+09,2.208087e+10,3.310130e+10,4.081142e+09,2.435920e+10,3.773112e+10
2,000004.SZ,计算机,2.080828e+06,4.696894e+06,6.829496e+06,1.558349e+06,2.517899e+06,5.536555e+06,3.720557e+06,5.989674e+06,...,-5.846578e+06,4.403971e+06,3.489948e+06,6.051302e+07,4.846545e+06,6.768830e+06,3.444398e+07,-1.309889e+07,-4.101779e+07,-6.634204e+07
3,000005.SZ,电力及公用事业,-6.291033e+06,2.712336e+07,1.365665e+07,-1.222558e+07,-2.925819e+07,-1.338760e+07,-9.235390e+06,-2.245686e+07,...,7.198863e+07,-2.230184e+07,-3.336571e+07,-2.759660e+07,8.595700e+06,2.005893e+08,2.032864e+08,8.476993e+05,3.500718e+06,7.114178e+06
4,000006.SZ,房地产,7.129346e+07,1.888583e+08,3.602888e+08,1.187848e+08,3.855712e+08,4.844263e+08,8.335351e+07,3.547500e+08,...,5.909587e+08,1.829440e+08,3.056448e+08,6.234316e+08,3.655433e+08,5.056996e+08,7.354123e+08,5.677890e+07,3.164525e+08,2.806002e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5005,688799.SH,医药,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.761440e+07,2.915974e+07,6.388089e+07,1.025185e+08,3.881189e+07,7.983247e+07,1.203837e+08,3.984571e+07,9.557748e+07,1.375809e+08
5006,688800.SH,电子,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.999367e+06,4.943665e+07,7.012116e+07,1.874455e+07,4.223445e+07,7.987381e+07,6.377653e+07,1.431755e+08,2.241252e+08
5007,688819.SH,电力设备及新能源,NaN,2.930077e+08,5.574871e+08,2.226396e+08,4.854227e+08,7.762266e+08,1.371183e+08,1.226802e+08,...,1.132626e+09,4.031860e+08,9.755104e+08,1.845066e+09,5.226400e+08,8.412500e+08,1.307616e+09,4.854114e+08,8.622961e+08,1.677084e+09
5008,688981.SH,电子,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.867820e+08,4.486887e+08,1.424889e+09,3.314351e+09,9.197230e+08,5.074229e+09,7.555459e+09,3.650392e+09,7.866118e+09,1.177550e+10


# Get nonfreq in Quarter 

In [4]:
df = df_nonfreq.set_index(["SECU_CODE","INDUSTRY"])
df

,,2011Q1,2011Q2,2011Q3,2012Q1,2012Q2,2012Q3,2013Q1,2013Q2,2013Q3,2014Q1,...,2019Q3,2020Q1,2020Q2,2020Q3,2021Q1,2021Q2,2021Q3,2022Q1,2022Q2,2022Q3
SECU_CODE,INDUSTRY,,,,,,,,,,,,,,,,,,,,,
000001.SZ,银行,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000002.SZ,房地产,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000004.SZ,计算机,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.537224e+05,6.818958e+05,8.521211e+05,9.688393e+04,6.156794e+05,7.568202e+05,4.529482e+05,4.701978e+05,4.821108e+05
000005.SZ,电力及公用事业,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.268691e+06,4.444005e+06,NaN,3.880938e+05,5.113436e+05,5.816464e+04,1.773372e+06,2.271582e+06
000006.SZ,房地产,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200000.0,2940840.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688799.SH,医药,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.597428e+05,7.897600e+06,NaN,1.162304e+07,1.391226e+07,1.640907e+07,1.249457e+06,5.596206e+06,9.023629e+06
688800.SH,电子,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.238674e+06,6.010483e+06,NaN,1.369872e+06,NaN,4.624037e+06,9.809686e+06,1.323329e+07,1.500273e+07
688819.SH,电力设备及新能源,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.783950e+08,NaN,2.248161e+08,2.521462e+08,1.101094e+08,2.387639e+08,3.099354e+08,1.027381e+08,2.649832e+08,3.804919e+08


In [5]:
df_nonfreq = gf.get_quarter(df)
df_nonfreq

,,2011Q1,2011Q2,2011Q3,2012Q1,2012Q2,2012Q3,2013Q1,2013Q2,2013Q3,2014Q1,...,2019Q3,2020Q1,2020Q2,2020Q3,2021Q1,2021Q2,2021Q3,2022Q1,2022Q2,2022Q3
SECU_CODE,INDUSTRY,,,,,,,,,,,,,,,,,,,,,
000001.SZ,银行,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000002.SZ,房地产,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000004.SZ,计算机,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.537224e+05,2.281734e+05,1.702253e+05,9.688393e+04,5.187955e+05,1.411407e+05,4.529482e+05,1.724966e+04,1.191296e+04
000005.SZ,电力及公用事业,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.134346e+06,1.753143e+05,NaN,1.940469e+05,1.232498e+05,5.816464e+04,1.715207e+06,4.982102e+05
000006.SZ,房地产,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100000.0,2740840.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688799.SH,医药,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.597428e+05,6.937857e+06,NaN,1.162304e+07,2.289221e+06,2.496801e+06,1.249457e+06,4.346749e+06,3.427423e+06
688800.SH,电子,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.238674e+06,3.771809e+06,NaN,1.369872e+06,-6.849362e+05,NaN,9.809686e+06,3.423600e+06,1.769442e+06
688819.SH,电力设备及新能源,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11066640.37,NaN,1.124080e+08,2.733011e+07,1.101094e+08,1.286545e+08,7.117148e+07,1.027381e+08,1.622451e+08,1.155087e+08


# Get Sum of Nonfreq by Industry

In [6]:
df_ind_sum = df_nonfreq.groupby("INDUSTRY").sum()
df_ind_sum = df_ind_sum.stack().reset_index()
df_ind_sum = df_ind_sum.rename(columns={"level_1": "QUARTER", 0: "SUM_NONFREQ"})
df_ind_sum = df_ind_sum.set_index(["INDUSTRY", "QUARTER"])
df_ind_sum

SUM_NONFREQ
INDUSTRY QUARTER              
交通运输     2011Q1   2.424421e+08
         2011Q2   5.731104e+08
         2011Q3   5.774365e+08
         2012Q1   8.498420e+08
         2012Q2   2.255177e+08
...                        ...
食品饮料     2021Q2   1.380475e+09
         2021Q3   1.256142e+09
         2022Q1   1.564331e+09
         2022Q2   1.334396e+09
         2022Q3   1.154834e+09

[1080 rows x 1 columns]

# $Support_i$ = $subsidy_i$/sum($subsidy_s$)

In [7]:
df = df_nonfreq.stack()
df = df.reset_index()
df = df.rename(columns={"level_2": "QUARTER", 0: "NONFREQ_SUB"})
df

,SECU_CODE,INDUSTRY,QUARTER,NONFREQ_SUB
0,000001.SZ,银行,2018Q1,14000000.00
1,000001.SZ,银行,2018Q2,-7000000.00
2,000004.SZ,计算机,2016Q2,2000.00
3,000004.SZ,计算机,2018Q1,3530000.00
4,000004.SZ,计算机,2018Q3,8000.00
...,...,...,...,...
107237,689009.SH,机械,2021Q2,3108619.80
107238,689009.SH,机械,2021Q3,3788935.25
107239,689009.SH,机械,2022Q1,1859192.19
107240,689009.SH,机械,2022Q2,10009062.96


In [8]:
df = df.set_index(["INDUSTRY", "QUARTER"])  # synchronize index
df

SECU_CODE  NONFREQ_SUB
INDUSTRY QUARTER                        
银行       2018Q1   000001.SZ  14000000.00
         2018Q2   000001.SZ  -7000000.00
计算机      2016Q2   000004.SZ      2000.00
         2018Q1   000004.SZ   3530000.00
         2018Q3   000004.SZ      8000.00
...                     ...          ...
机械       2021Q2   689009.SH   3108619.80
         2021Q3   689009.SH   3788935.25
         2022Q1   689009.SH   1859192.19
         2022Q2   689009.SH  10009062.96
         2022Q3   689009.SH     60073.80

[107242 rows x 2 columns]

In [9]:
df_nonfreq = pd.merge(
    left=df, right=df_ind_sum, left_index=True, right_index=True
)  # merge the Dataframe to make nonfreq and industry sum in the same Dataframe
df_nonfreq = df_nonfreq.reset_index()
df_nonfreq = df_nonfreq.sort_values(["QUARTER", "SECU_CODE"])
df_nonfreq["F_SUPPORT"] = df_nonfreq["NONFREQ_SUB"] / df_nonfreq["SUM_NONFREQ"]
df_nonfreq = df_nonfreq.set_index(["QUARTER", "SECU_CODE", "INDUSTRY"])
df_nonfreq

NONFREQ_SUB   SUM_NONFREQ  F_SUPPORT
QUARTER SECU_CODE INDUSTRY                                       
2011Q1  000009.SZ 综合        2.126029e+06  3.469076e+07   0.061285
        000012.SZ 建材        2.477677e+07  1.613758e+08   0.153535
        000016.SZ 家电        1.827360e+07  2.603773e+08   0.070181
        000019.SZ 农林牧渔      2.684000e+05  1.002543e+08   0.002677
        000020.SZ 电子        4.000000e+04  3.146386e+08   0.000127
...                                  ...           ...        ...
2022Q3  688799.SH 医药        3.427423e+06  3.246630e+09   0.001056
        688800.SH 电子        1.769442e+06  7.211184e+09   0.000245
        688819.SH 电力设备及新能源  1.155087e+08  4.544736e+09   0.025416
        688981.SH 电子        6.406340e+08  7.211184e+09   0.088839
        689009.SH 机械        6.007380e+04  3.959157e+09   0.000015

[107242 rows x 3 columns]

# Get $profit$ in quarter

In [10]:
df = df_profit.set_index(["SECU_CODE", "INDUSTRY"])
df_profit = gf.get_quarter(df)
df_profit

,,2011Q1,2011Q2,2011Q3,2012Q1,2012Q2,2012Q3,2013Q1,2013Q2,2013Q3,2014Q1,...,2019Q3,2020Q1,2020Q2,2020Q3,2021Q1,2021Q2,2021Q3,2022Q1,2022Q2,2022Q3
SECU_CODE,INDUSTRY,,,,,,,,,,,,,,,,,,,,,
000001.SZ,银行,3.069099e+09,2.996747e+09,3.858597e+09,4.477351e+09,4.401298e+09,4.533465e+09,4.718000e+09,5.188000e+09,5.479000e+09,6.697000e+09,...,1.067300e+10,1.095800e+10,6.629000e+09,1.123900e+10,1.287400e+10,9.467000e+09,1.444600e+10,1.618400e+10,1.159900e+10,1.832900e+10
000002.SZ,房地产,1.674455e+09,2.777354e+09,1.198387e+09,2.058143e+09,3.990529e+09,2.117311e+09,2.394886e+09,4.738527e+09,2.867622e+09,2.039198e+09,...,1.409881e+10,4.795679e+09,2.165477e+10,1.538755e+10,4.108071e+09,1.797280e+10,1.102043e+10,4.081142e+09,2.027806e+10,1.337192e+10
000004.SZ,计算机,2.080828e+06,2.616066e+06,2.132602e+06,1.558349e+06,9.595499e+05,3.018656e+06,3.720557e+06,2.269117e+06,4.276827e+06,-2.974611e+06,...,-8.795067e+06,4.403971e+06,-9.140229e+05,5.702308e+07,4.846545e+06,1.922285e+06,2.767515e+07,-1.309889e+07,-2.791890e+07,-2.532426e+07
000005.SZ,电力及公用事业,-6.291033e+06,3.341440e+07,-1.346671e+07,-1.222558e+07,-1.703261e+07,1.587058e+07,-9.235390e+06,-1.322147e+07,-6.543229e+06,-8.682228e+06,...,-1.663681e+07,-2.230184e+07,-1.106386e+07,5.769103e+06,8.595700e+06,1.919936e+08,2.697103e+06,8.476993e+05,2.653019e+06,3.613460e+06
000006.SZ,房地产,7.129346e+07,1.175649e+08,1.714304e+08,1.187848e+08,2.667864e+08,9.885507e+07,8.335351e+07,2.713965e+08,1.207738e+08,4.036137e+07,...,3.318325e+08,1.829440e+08,1.227008e+08,3.177867e+08,3.655433e+08,1.401563e+08,2.297127e+08,5.677890e+07,2.596736e+08,-3.585225e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688799.SH,医药,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.915974e+07,3.472115e+07,3.863758e+07,3.881189e+07,4.102057e+07,4.055120e+07,3.984571e+07,5.573177e+07,4.200342e+07
688800.SH,电子,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.999367e+06,3.943728e+07,2.068451e+07,1.874455e+07,2.348991e+07,3.763935e+07,6.377653e+07,7.939895e+07,8.094975e+07
688819.SH,电力设备及新能源,NaN,1.465038e+08,2.644794e+08,2.226396e+08,2.627831e+08,2.908038e+08,1.371183e+08,-1.443808e+07,1.287562e+08,6.622459e+07,...,5.615807e+08,4.031860e+08,5.723244e+08,8.695559e+08,5.226400e+08,3.186100e+08,4.663662e+08,4.854114e+08,3.768847e+08,8.147876e+08


In [11]:
df = df_profit.stack()
df =df.reset_index()
df =df.rename(columns = {"level_2": "QUARTER",0: "PROFIT"})
df =df.set_index(["QUARTER","SECU_CODE","INDUSTRY"]) # synchronize index
df

,,,PROFIT
QUARTER,SECU_CODE,INDUSTRY,
2011Q1,000001.SZ,银行,3.069099e+09
2011Q2,000001.SZ,银行,2.996747e+09
2011Q3,000001.SZ,银行,3.858597e+09
2012Q1,000001.SZ,银行,4.477351e+09
2012Q2,000001.SZ,银行,4.401298e+09
...,...,...,...
2021Q2,689009.SH,机械,3.214818e+08
2021Q3,689009.SH,机械,1.860397e+08
2022Q1,689009.SH,机械,7.554774e+07


In [12]:
df_nonfreq = pd.merge(
    left=df_nonfreq, right=df, left_index=True, right_index=True
)  # merge the dataframe to make nonfreq and profit in the same dataframe
df_nonfreq

NONFREQ_SUB   SUM_NONFREQ  F_SUPPORT  \
QUARTER SECU_CODE INDUSTRY                                          
2011Q1  000009.SZ 综合        2.126029e+06  3.469076e+07   0.061285   
        000012.SZ 建材        2.477677e+07  1.613758e+08   0.153535   
        000016.SZ 家电        1.827360e+07  2.603773e+08   0.070181   
        000019.SZ 农林牧渔      2.684000e+05  1.002543e+08   0.002677   
        000020.SZ 电子        4.000000e+04  3.146386e+08   0.000127   
...                                  ...           ...        ...   
2022Q3  688799.SH 医药        3.427423e+06  3.246630e+09   0.001056   
        688800.SH 电子        1.769442e+06  7.211184e+09   0.000245   
        688819.SH 电力设备及新能源  1.155087e+08  4.544736e+09   0.025416   
        688981.SH 电子        6.406340e+08  7.211184e+09   0.088839   
        689009.SH 机械        6.007380e+04  3.959157e+09   0.000015   

                                  PROFIT  
QUARTER SECU_CODE INDUSTRY                
2011Q1  000009.SZ 综合        1.768050e+08  
        000012.SZ 建材        5.429056e+08  
        000016.SZ 家电       -6.889821e+07  
        000019.SZ 农林牧渔     -5.513920e+06  
        000020.SZ 电子        3.771890e+06  
...                                  ...  
2022Q3  688799.SH 医药        4.200342e+07  
        688800.SH 电子        8.094975e+07  
        688819.SH 电力设备及新能源  8.147876e+08  
        688981.SH 电子        3.909379e+09  
        689009.SH 机械        1.828479e+08  

[107193 rows x 4 columns]

# dependence = subsidy/profit

In [13]:
df_nonfreq["DEPENDENCE"] = df_nonfreq["NONFREQ_SUB"] / df_nonfreq["PROFIT"]
# subsidy > profit : dependence = 1
# other: original value
df_nonfreq["F_DEPENDENCE"] = df_nonfreq.DEPENDENCE
df_nonfreq["F_DEPENDENCE"] = df_nonfreq.F_DEPENDENCE.mask(
    (df_nonfreq.NONFREQ_SUB > df_nonfreq.PROFIT), 1
)
df_nonfreq

NONFREQ_SUB   SUM_NONFREQ  F_SUPPORT  \
QUARTER SECU_CODE INDUSTRY                                          
2011Q1  000009.SZ 综合        2.126029e+06  3.469076e+07   0.061285   
        000012.SZ 建材        2.477677e+07  1.613758e+08   0.153535   
        000016.SZ 家电        1.827360e+07  2.603773e+08   0.070181   
        000019.SZ 农林牧渔      2.684000e+05  1.002543e+08   0.002677   
        000020.SZ 电子        4.000000e+04  3.146386e+08   0.000127   
...                                  ...           ...        ...   
2022Q3  688799.SH 医药        3.427423e+06  3.246630e+09   0.001056   
        688800.SH 电子        1.769442e+06  7.211184e+09   0.000245   
        688819.SH 电力设备及新能源  1.155087e+08  4.544736e+09   0.025416   
        688981.SH 电子        6.406340e+08  7.211184e+09   0.088839   
        689009.SH 机械        6.007380e+04  3.959157e+09   0.000015   

                                  PROFIT  DEPENDENCE  F_DEPENDENCE  
QUARTER SECU_CODE INDUSTRY                                          
2011Q1  000009.SZ 综合        1.768050e+08    0.012025      0.012025  
        000012.SZ 建材        5.429056e+08    0.045637      0.045637  
        000016.SZ 家电       -6.889821e+07   -0.265226      1.000000  
        000019.SZ 农林牧渔     -5.513920e+06   -0.048677      1.000000  
        000020.SZ 电子        3.771890e+06    0.010605      0.010605  
...                                  ...         ...           ...  
2022Q3  688799.SH 医药        4.200342e+07    0.081599      0.081599  
        688800.SH 电子        8.094975e+07    0.021859      0.021859  
        688819.SH 电力设备及新能源  8.147876e+08    0.141765      0.141765  
        688981.SH 电子        3.909379e+09    0.163871      0.163871  
        689009.SH 机械        1.828479e+08    0.000329      0.000329  

[107193 rows x 6 columns]

In [14]:
# delete redundant columns
df_nonfreq = df_nonfreq.drop(
    ["NONFREQ_SUB", "SUM_NONFREQ", "PROFIT", "DEPENDENCE"], axis=1
)
df_nonfreq

F_SUPPORT  F_DEPENDENCE
QUARTER SECU_CODE INDUSTRY                         
2011Q1  000009.SZ 综合         0.061285      0.012025
        000012.SZ 建材         0.153535      0.045637
        000016.SZ 家电         0.070181      1.000000
        000019.SZ 农林牧渔       0.002677      1.000000
        000020.SZ 电子         0.000127      0.010605
...                               ...           ...
2022Q3  688799.SH 医药         0.001056      0.081599
        688800.SH 电子         0.000245      0.021859
        688819.SH 电力设备及新能源   0.025416      0.141765
        688981.SH 电子         0.088839      0.163871
        689009.SH 机械         0.000015      0.000329

[107193 rows x 2 columns]

# Rank F_SUPPORT

In [15]:
df_nonfreq["F_SUPPORT"] = df_nonfreq.groupby(by = "QUARTER").rank(axis = 0)["F_SUPPORT"]
df_nonfreq

F_SUPPORT  F_DEPENDENCE
QUARTER SECU_CODE INDUSTRY                         
2011Q1  000009.SZ 综合           1210.0      0.012025
        000012.SZ 建材           1275.0      0.045637
        000016.SZ 家电           1225.0      1.000000
        000019.SZ 农林牧渔          482.0      1.000000
        000020.SZ 电子             81.5      0.010605
...                               ...           ...
2022Q3  688799.SH 医药           2201.0      0.081599
        688800.SH 电子            855.0      0.021859
        688819.SH 电力设备及新能源     4460.0      0.141765
        688981.SH 电子           4664.0      0.163871
        689009.SH 机械            120.0      0.000329

[107193 rows x 2 columns]

# Standardization

In [16]:
df =df_nonfreq
mean =df.groupby(by = "QUARTER").mean()
std = df.groupby(by="QUARTER").std()

df = gf.get_standard(df,mean,std)
df

,SECU_CODE,INDUSTRY,F_SUPPORT,F_DEPENDENCE,F_SUPPORT_mean,F_DEPENDENCE_mean,F_SUPPORT_std,F_DEPENDENCE_std
QUARTER,,,,,,,,
2011Q1,000009.SZ,综合,1.458430,-0.543251,657.0,0.184724,379.174750,0.317901
2011Q1,000012.SZ,建材,1.629855,-0.437518,657.0,0.184724,379.174750,0.317901
2011Q1,000016.SZ,家电,1.497990,2.564561,657.0,0.184724,379.174750,0.317901
2011Q1,000019.SZ,农林牧渔,-0.461529,2.564561,657.0,0.184724,379.174750,0.317901
2011Q1,000020.SZ,电子,-1.517770,-0.547717,657.0,0.184724,379.174750,0.317901
...,...,...,...,...,...,...,...,...
2022Q3,688799.SH,医药,-0.114306,-0.570632,2356.5,0.310671,1360.381564,0.401436
2022Q3,688800.SH,电子,-1.103734,-0.719448,2356.5,0.310671,1360.381564,0.401436
2022Q3,688819.SH,电力设备及新能源,1.546257,-0.420753,2356.5,0.310671,1360.381564,0.401436


In [17]:
df_nonfreq = df.drop(
    ["F_SUPPORT_mean", "F_DEPENDENCE_mean", "F_SUPPORT_std", "F_DEPENDENCE_std"], axis=1
)
df_nonfreq

,SECU_CODE,INDUSTRY,F_SUPPORT,F_DEPENDENCE
QUARTER,,,,
2011Q1,000009.SZ,综合,1.458430,-0.543251
2011Q1,000012.SZ,建材,1.629855,-0.437518
2011Q1,000016.SZ,家电,1.497990,2.564561
2011Q1,000019.SZ,农林牧渔,-0.461529,2.564561
2011Q1,000020.SZ,电子,-1.517770,-0.547717
...,...,...,...,...
2022Q3,688799.SH,医药,-0.114306,-0.570632
2022Q3,688800.SH,电子,-1.103734,-0.719448
2022Q3,688819.SH,电力设备及新能源,1.546257,-0.420753


# Exclude Outlier

In [18]:
df_nonfreq[df_nonfreq.F_SUPPORT > 3] = np.nan
df_nonfreq[df_nonfreq.F_SUPPORT < -3] = np.nan
df_nonfreq[df_nonfreq.F_DEPENDENCE > 3] = np.nan
df_nonfreq[df_nonfreq.F_DEPENDENCE < -3] = np.nan
df_nonfreq

,SECU_CODE,INDUSTRY,F_SUPPORT,F_DEPENDENCE
QUARTER,,,,
2011Q1,000009.SZ,综合,1.458430,-0.543251
2011Q1,000012.SZ,建材,1.629855,-0.437518
2011Q1,000016.SZ,家电,1.497990,2.564561
2011Q1,000019.SZ,农林牧渔,-0.461529,2.564561
2011Q1,000020.SZ,电子,-1.517770,-0.547717
...,...,...,...,...
2022Q3,688799.SH,医药,-0.114306,-0.570632
2022Q3,688800.SH,电子,-1.103734,-0.719448
2022Q3,688819.SH,电力设备及新能源,1.546257,-0.420753


# F_SUBSIDY = F_SUPPORT - F_DEPENDENCE

In [19]:
df_nonfreq["F_SUBSIDY"] = df_nonfreq["F_SUPPORT"] - df_nonfreq["F_DEPENDENCE"]
df_nonfreq

,SECU_CODE,INDUSTRY,F_SUPPORT,F_DEPENDENCE,F_SUBSIDY
QUARTER,,,,,
2011Q1,000009.SZ,综合,1.458430,-0.543251,2.001681
2011Q1,000012.SZ,建材,1.629855,-0.437518,2.067373
2011Q1,000016.SZ,家电,1.497990,2.564561,-1.066571
2011Q1,000019.SZ,农林牧渔,-0.461529,2.564561,-3.026089
2011Q1,000020.SZ,电子,-1.517770,-0.547717,-0.970052
...,...,...,...,...,...
2022Q3,688799.SH,医药,-0.114306,-0.570632,0.456326
2022Q3,688800.SH,电子,-1.103734,-0.719448,-0.384286
2022Q3,688819.SH,电力设备及新能源,1.546257,-0.420753,1.967011


In [20]:
df_nonfreq = df_nonfreq.reset_index().sort_values(by = ["QUARTER", "SECU_CODE"])
df_nonfreq = df_nonfreq.set_index("QUARTER")

df_nonfreq

,SECU_CODE,INDUSTRY,F_SUPPORT,F_DEPENDENCE,F_SUBSIDY
QUARTER,,,,,
2011Q1,000009.SZ,综合,1.458430,-0.543251,2.001681
2011Q1,000012.SZ,建材,1.629855,-0.437518,2.067373
2011Q1,000016.SZ,家电,1.497990,2.564561,-1.066571
2011Q1,000019.SZ,农林牧渔,-0.461529,2.564561,-3.026089
2011Q1,000020.SZ,电子,-1.517770,-0.547717,-0.970052
...,...,...,...,...,...
2022Q3,688799.SH,医药,-0.114306,-0.570632,0.456326
2022Q3,688800.SH,电子,-1.103734,-0.719448,-0.384286
2022Q3,688819.SH,电力设备及新能源,1.546257,-0.420753,1.967011


# Write File

In [21]:
df_nonfreq.to_csv("cleaned_data/cleaned_nonfreq.csv")